In [90]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [346]:
class BasicBlock(nn.Module): #for resnet18, 34
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):  
        super().__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module): #for resnet50+
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes: #why this terms? because in the first block of each layers the in_planes is not equal to planes*expansion ,so we cannot add shortcut to our output because of the difference shape
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x): #bottleneck's forward will work along with resnet's forward for example when layer1 in resnet be called then it will work process in bottleneck
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2) #64 > 32 (because stride = 2, its like /2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2) # 32 > 16
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2) #16 > 4 and then use angpool = 4 = 4/4 = 1
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1) #this stride gonna set conv2 of each layers to stride = stride but in conv1 and 3, stride is fix = 1
        layers = []
        for stride in strides: #this very genius and kinda tricky 
            layers.append(block(self.in_planes, planes, stride)) #for example the end of block1 in layer1 is 256(64*4) then in the next loop we set first in_planes to 64*4 to start of block2 because it still not change but after this it will change to new planes * expansion (128*4)
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out) #64 (h, w)
        out = self.layer2(out) #32
        out = self.layer3(out) #16
        out = self.layer4(out) # 4
        #print(out.shape) 
        out = F.avg_pool2d(out, 4) #its 4 because in this case our out is 4,4 (h, w) so we gotta make it to 1,1 (4/4)
        out = out.view(out.size(0), -1) #its out.size(0) why? that is batch, yes but we need to let h, w be 1 too, so if isn't 1 it'll be error and we will know that
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34(): #if you're wondering why this is same as resnet50, it's because resnet34 use basic block, expansion = 1, not bottle block expansion=4
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


In [348]:
stride = 2
strides = [stride] + [1]*3
strides

[2, 1, 1, 1]

In [350]:
device = "cuda"
model = ResNet34().to(device)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [352]:
trans = transforms.Compose([
    transforms.TrivialAugmentWide(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])
train_data = datasets.CIFAR10(root="../data", download=False, train=True, transform=trans)
test_data = datasets.CIFAR10(root="../data", download=False, train=False, transform=transforms.ToTensor())

In [353]:
batch_size = 20
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_dataloader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

In [354]:
from torch.optim import lr_scheduler
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05, weight_decay=0.0001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.2)
epochs = 15

In [358]:
from sklearn.metrics import accuracy_score
from timeit import default_timer as timer
start = timer()
for epoch in range(epochs):
    loss_train = 0
    for x, y in train_dataloader:
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss = criterion(y_pred, y)
        loss_train += loss
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_value_(model.parameters(), 2.0)
        optimizer.step()
    scheduler.step()

    loss_train /= len(train_dataloader)
    loss_test, acc =  0, 0
    model.eval()
    with torch.inference_mode():
        for x, y in test_dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss_test += criterion(y_pred, y)
            acc += accuracy_score(torch.softmax(y_pred, dim=1).argmax(dim=1).cpu(), y.cpu())
        loss_test /= len(test_dataloader)
        acc /= len(test_dataloader)
    print(f"epoch : {epoch}, loss : {loss_train:.4f}, loss_test : {loss_test:.4f}, acc : {(acc*100):.3f}, lr : {optimizer.state_dict()['param_groups'][0]['lr']:.5f}")
end = timer()
round(end-start), "sec"

epoch : 0, loss : 1.9383, loss_test : 1.3327, acc : 51.660, lr : 0.05000
epoch : 1, loss : 1.7509, loss_test : 1.3874, acc : 49.110, lr : 0.05000
epoch : 2, loss : 1.3850, loss_test : 1.0174, acc : 63.210, lr : 0.05000
epoch : 3, loss : 1.1972, loss_test : 0.8659, acc : 69.850, lr : 0.05000
epoch : 4, loss : 1.0722, loss_test : 0.9729, acc : 66.840, lr : 0.05000
epoch : 5, loss : 0.9682, loss_test : 0.6247, acc : 78.240, lr : 0.05000
epoch : 6, loss : 0.9008, loss_test : 0.5983, acc : 79.530, lr : 0.05000
epoch : 7, loss : 0.8313, loss_test : 0.5416, acc : 81.890, lr : 0.05000
epoch : 8, loss : 0.7862, loss_test : 0.5967, acc : 80.160, lr : 0.05000
epoch : 9, loss : 0.7478, loss_test : 0.4830, acc : 83.310, lr : 0.01000
epoch : 10, loss : 0.6019, loss_test : 0.4087, acc : 86.200, lr : 0.01000
epoch : 11, loss : 0.5672, loss_test : 0.4039, acc : 86.540, lr : 0.01000
epoch : 12, loss : 0.5476, loss_test : 0.3831, acc : 87.030, lr : 0.01000
epoch : 13, loss : 0.5324, loss_test : 0.3741, a

(859, 'sec')